https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/scikit_learn_randomforest/Sklearn_on_SageMaker_end2end.ipynb

In [41]:
import os
import boto3
import re
import sagemaker
import pandas as pd
import json
import numpy as np
from sagemaker import get_execution_role
from sklearn.metrics import r2_score
import pprint

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sm_boto3 = boto3.client('sagemaker')
sess = sagemaker.Session()

bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/bmeajg31'
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region, bucket)

In [2]:
data = pd.read_csv('s3://sagemakerbmeironia/sagemaker_input_data/data.csv', sep=';')
data.set_index('date_ws', inplace=True)

In [3]:
s3 = boto3.resource('s3')
content_object = s3.Object('sagemakerbmeironia', 'sagemaker_input_data/names.json')
file_content = content_object.get()['Body'].read().decode('utf-8')
names_dict = json.loads(file_content)

# Filtrado de fondos

In [4]:
n = 10
filt_columns = data.columns.tolist()[:10]
filt_names = {v:k for k,v in names_dict.items() if str(v) in filt_columns}
data_filt = data[filt_columns]

In [5]:
check_equal_df = False
while not check_equal_df:
    data_pre = data_filt.copy()
    data_filt.fillna(method='ffill', inplace=True)
    check_equal_df = data_filt.equals(data_pre)

check_equal_df = False
while not check_equal_df:
    data_pre = data_filt.copy()
    data_filt.fillna(method='bfill', inplace=True)
    check_equal_df = data_filt.equals(data_pre)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [6]:
returns = np.log(data_filt).diff()

In [7]:
ret_f1 = returns.shift(-1)

In [8]:
ret_p1 = returns.shift(1)
ret_p1.columns = [elem+'_1' for elem in returns.columns.tolist()]
ret_p2 = returns.shift(2)
ret_p2.columns = [elem+'_2' for elem in returns.columns.tolist()]
ret_p3 = returns.shift(3)
ret_p3.columns = [elem+'_3' for elem in returns.columns.tolist()]
ret_p4 = returns.shift(4)
ret_p4.columns = [elem+'_4' for elem in returns.columns.tolist()]
ret_p5 = returns.shift(5)
ret_p5.columns = [elem+'_5' for elem in returns.columns.tolist()]

In [9]:
inputs = pd.concat([returns, ret_p1, ret_p2, ret_p3, ret_p4, ret_p5], axis=1).iloc[5+1:-1]
target = ret_f1.iloc[5+1:-1]

# División en train, test y split

In [10]:
perc_train = 0.7
perc_test = 0.2

In [11]:
train_x, train_y = inputs.iloc[:int(len(inputs)*perc_train)] , target.iloc[:int(len(inputs)*perc_train)]
test_x, test_y = inputs.iloc[int(len(inputs)*perc_train):int(len(inputs)*(perc_train+perc_test))] , target.iloc[int(len(inputs)*perc_train):int(len(inputs)*(perc_train+perc_test))]
val_x, val_y = inputs.iloc[int(len(inputs)*(perc_train+perc_test)):] , target.iloc[int(len(inputs)*(perc_train+perc_test)):]

# Persistimos los datos

In [12]:
with open('fund_names.json', 'w') as fp:
    json.dump(filt_names, fp)

In [13]:
train_x.to_csv('train_x.csv', index=False)
train_y.to_csv('train_y.csv', index=False)
test_x.to_csv('test_x.csv', index=False)
test_y.to_csv('test_y.csv', index=False)

In [14]:
names = sess.upload_data(
    path='fund_names.json', bucket=bucket,
    key_prefix='sagemaker/sklearncontainer')

train_x_path = sess.upload_data(
    path='train_x.csv', bucket=bucket,
    key_prefix='sagemaker/sklearncontainer')

train_y_path = sess.upload_data(
    path='train_y.csv', bucket=bucket,
    key_prefix='sagemaker/sklearncontainer')

test_x_path = sess.upload_data(
    path='test_x.csv', bucket=bucket,
    key_prefix='sagemaker/sklearncontainer')

test_y_path = sess.upload_data(
    path='test_y.csv', bucket=bucket,
    key_prefix='sagemaker/sklearncontainer')

# Generamos script

In [15]:
%%writefile script.py

import argparse
import joblib
import json
import os
import pprint

import boto3
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

import subprocess
import sys



# inference functions ---------------
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf



if __name__ =='__main__':
    
    subprocess.check_call([sys.executable, "-m", "pip", "install", "s3fs"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "fsspec"])
    

    print('extracting arguments')
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # to simplify the demo we don't use all sklearn RandomForest hyperparameters
    parser.add_argument('--n-estimators', type=int, default=10)

    # Data, model, and output directories
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))
    parser.add_argument('--x_train_file', type=str, default='train_x.csv')
    parser.add_argument('--y_train_file', type=str, default='train_y.csv')
    parser.add_argument('--x_test_file', type=str, default='test_x.csv')
    parser.add_argument('--y_test_file', type=str, default='test_y.csv')
    parser.add_argument('--names_file', type=str, default='fund_names.json')

    args, _ = parser.parse_known_args()


    if args.train == 'unused':
        print('reading data')
        x_train = pd.read_csv(args.x_train_file).values
        y_train = pd.read_csv(args.y_train_file).values
        x_test = pd.read_csv(args.x_test_file).values
        y_test_df = pd.read_csv(args.y_test_file)
        y_test = y_test_df.values
        columns = y_test_df.columns.tolist()
        
        print('reading fund names')
        s3 = boto3.resource('s3')
        a = args.names_file
        bucket = a.split('//')[1].split('/')[0]
        file = a.split(bucket)[1][1:]
        content_object = s3.Object(bucket, file)
        file_content = content_object.get()['Body'].read().decode('utf-8')
        names_dict = json.loads(file_content)
        
    else:
        print('reading data')
        x_train = pd.read_csv(os.path.join(args.train, args.x_train_file)).values
        y_train = pd.read_csv(os.path.join(args.train, args.y_train_file)).values
        x_test = pd.read_csv(os.path.join(args.test, args.x_test_file)).values
        y_test_df = pd.read_csv(os.path.join(args.test, args.y_test_file))
        y_test = y_test_df.values
        columns = y_test_df.columns.tolist()

        print('reading fund names')
        json_file = open(os.path.join(args.train, args.names_file))
        json_str = json_file.read()
        names_dict = json.loads(json_str)


    print('training model')
    model = RandomForestRegressor(
        n_estimators=args.n_estimators)
    
    model.fit(x_train, y_train)

    # print abs error
    print('validating model')
    pred = model.predict(x_test)
    results_dict = {}
    for index, elem in enumerate(columns):
        score = r2_score(y_test[:,index], pred[:,index])
        results_dict[names_dict[elem]] = score

    pprint.pprint(results_dict)

    # persist model
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print('model persisted at ' + path)

Overwriting script.py


# Test local

In [16]:
! python script.py --n-estimators 100 \
                   --model-dir ./ \
                   --train ./ \
                   --test ./ \

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
extracting arguments
reading data
reading fund names
training model
validating model
{'"FRANKLIN BIOTECHNOLOGY DISCOVERY ""I"""': -0.06535253419185749,
 '"FRANKLIN EURO HIGH YIELD ""I"" (EUR) ACC"': 0.0637345283567381,
 '"FRANKLIN EURO HIGH YIELD ""I"" (EUR) INC"': -0.1003302335526628,
 '"FRANKLIN EUROPEAN GROWTH ""I"""': -0.00852899336990709,
 '"FRANKLIN GLOBAL REAL ESTATE ""I"" ACC"': -0.046054955864316405,
 '"FRANKLIN INCOME ""I"""': -0.01096552349631752,
 '"FRANKLIN INDIA ""I"" (USD)"': -0.0241379230462877,
 '"JAN HEN GLOBAL EQUITY ""R"" (GBP) ACC"': -0.050897620215208494,
 '"JAN HEN GLOBAL EQUITY ""R"" (USD) ACC"': -0.03196791038012048,
 '"JAN HEN UK ABSOLUTE RETURN ""R"" (GBP) ACC"': -0.02048511964615174}
model persisted at ./mode

# Entranamos en Sagemaker

In [17]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = '0.23-1'

sklearn_estimator = SKLearn(
    entry_point='script.py',
    role = get_execution_role(),
    train_instance_count=1,
    train_instance_type='ml.m5.large',
    framework_version=FRAMEWORK_VERSION,
    base_job_name='rf-scikit',
    hyperparameters = {'n-estimators': 100,
                       'train':'unused',
                       'test':'unused',
                       'x_train_file': train_x_path,
                       'y_train_file': train_y_path,
                       'x_test_file': test_x_path,
                       'y_test_file': test_y_path,
                       'names_file': names})

In [18]:
sklearn_estimator.fit( wait=False)

# Desplegar un endpoint

In [19]:
sklearn_estimator.latest_training_job.wait()
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name)['ModelArtifacts']['S3ModelArtifacts']

print('Model artifact persisted at ' + artifact)

2020-10-29 21:32:00 Starting - Starting the training job...
2020-10-29 21:32:02 Starting - Launching requested ML instances......
2020-10-29 21:33:04 Starting - Preparing the instances for training......
2020-10-29 21:34:28 Downloading - Downloading input data
2020-10-29 21:34:28 Training - Downloading the training image....2020-10-29 21:35:06,250 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
2020-10-29 21:35:06,252 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-29 21:35:06,261 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-10-29 21:35:06,609 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-29 21:35:12,862 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-29 21:35:12,873 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-29 21:35:12,882 sagem

In [20]:
from sagemaker.sklearn.model import SKLearnModel

model = SKLearnModel(
    model_data=artifact,
    role=get_execution_role(),
    entry_point='script.py',
    framework_version=FRAMEWORK_VERSION)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [21]:
predictor = model.deploy(
    instance_type='ml.t2.medium',
    initial_instance_count=1)

-------------------!

# Predicciones

In [151]:
runtime = boto3.client('sagemaker-runtime')

In [152]:
response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint,
    Body=val_x.to_csv(header=False, index=False).encode('utf-8'),
    ContentType='text/csv')

In [153]:
response

{'ResponseMetadata': {'RequestId': '5aed36bc-be87-4e1b-bba7-fabaeaa7c415',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5aed36bc-be87-4e1b-bba7-fabaeaa7c415',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Thu, 29 Oct 2020 22:11:36 GMT',
   'content-type': 'application/json',
   'content-length': '65522'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7f776fe0c518>}

In [154]:
response['Body'].read()

b'[[0.002057374240859191, 0.0017837609048560199, 0.00037226514160447867, 0.0032458769973521973, 0.0007855385439955942, 0.00086642939543913, 0.000698064864798984, 0.0006813820432752804, 0.0007828945417060452, 0.0022711882470623657], [-0.003005360865975282, -0.0013256295768748383, -0.00046601713234225575, -0.0029021996261501216, -0.001183400481327279, -0.0009291875848658959, 0.00018068508665450623, -0.0004015209820539979, -0.00023859416850426473, -0.0015042369859764504], [-0.002483394983098104, -0.0014434790509766215, -0.0006296634075213237, 1.6775700837992242e-05, -0.0019513146653101954, -0.0013257467963799963, -0.00036574210673335995, -0.00033752268904342576, -0.0005375662115935632, -0.002166926789591619], [0.0010120592183098488, 0.00023397347438177234, 3.1475785592520695e-05, -0.0005719440755806062, 0.0006462216919083375, -0.0002628688846090711, 0.0005619744908116831, 0.0005216647308737343, -7.407261637463421e-05, 0.0007875260113436733], [0.0014474441525640658, 2.752244592914894e-05, 

# Borrar el endpoint

In [155]:
sm_boto3.delete_endpoint(EndpointName=predictor.endpoint)

{'ResponseMetadata': {'RequestId': 'd0340a98-3a84-40b4-ab73-076e1f25f746',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd0340a98-3a84-40b4-ab73-076e1f25f746',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 29 Oct 2020 22:11:44 GMT'},
  'RetryAttempts': 0}}